<a href="https://colab.research.google.com/github/ko-syo-noyome/Measurement-engineering-class/blob/main/create-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from PIL import Image
import os, glob #特定のパターンにマッチするファイルを取得することができる。：かたまり
import numpy as np
from sklearn import model_selection


classes=["ten","one-hundred","five-hundred"]
num_classes=len(classes)

for index, classlabel in enumerate(classes):
  photos_dir="drive/MyDrive/Measurement-engineering/"+classlabel
  print(photos_dir)

drive/MyDrive/Measurement-engineering/ten
drive/MyDrive/Measurement-engineering/one-hundred
drive/MyDrive/Measurement-engineering/five-hundred


In [3]:
classes=["ten","one-hundred","five-hundred"]

num_classes=len(classes)
image_size=100

# #画像の読み込み
X=[]
Y=[]

for index, classlabel in enumerate(classes):
  photos_dir="drive/MyDrive/Measurement-engineering/"+classlabel

  files=glob.glob(photos_dir+"/*.jpg")
  #print(files)
  
  for i,file in enumerate(files):
    #print(i)
    if i >=200: break
    image=Image.open(file)
    image=image.convert("RGB")
    image=image.resize((image_size,image_size))
    data=np.asarray(image)
    X.append(data)
    Y.append(index)

#tensorflowが読み込みやすいnumpyの配列にする
X=np.array(X)
Y=np.array(Y)





In [4]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,Y)
xy=(X_train,X_test,y_train,y_test)
np.save("drive/MyDrive/Measurement-engineering/model.npy",xy)


/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np
import tensorflow
import keras
import datetime


classes=["ten","one-hundred","five-hundred"]
num_classes=len(classes)
image_size=100


# main 関数

def model_train(X,y):
    model=Sequential()
    model.add(Conv2D(32,(3,3),padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    opt = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

    model.compile(loss='categorical_crossentropy',
                        optimizer=opt,metrics=['accuracy'])


    model.fit(X,y,batch_size=32,epochs=100)

    model.save('drive/MyDrive/Measurement-engineering/mefa-model.h5')

    return model


def main():
    X_train,X_test,y_train,y_test=np.load("drive/MyDrive/Measurement-engineering/model.npy",allow_pickle=True)

    print(X_train.shape)
   
    X_train=X_train.astype("float")/256
    X_test=X_test.astype("float")/256

    y_train=np_utils.to_categorical(y_train, num_classes)
    y_test=np_utils.to_categorical(y_test,num_classes)

    model=model_train(X_train,y_train)
    model_eval(model,X_test,y_test)


    return 

def model_eval(model,X,y):
    scores=model.evaluate(X,y,verbose=1)
    print('Test Loss:',scores[0])
    print('Test Accuracy:',scores[1])


if __name__=="__main__":
    main()




(45, 100, 100, 3)
Epoch 1/100
2/2 [==============================] - 4s 840ms/step - loss: 1.1149 - accuracy: 0.4222
Epoch 2/100
2/2 [==============================] - 3s 837ms/step - loss: 1.3205 - accuracy: 0.4000
Epoch 3/100
2/2 [==============================] - 3s 804ms/step - loss: 0.9419 - accuracy: 0.5778
Epoch 4/100
2/2 [==============================] - 3s 803ms/step - loss: 0.9488 - accuracy: 0.4889
Epoch 5/100
2/2 [==============================] - 3s 804ms/step - loss: 0.8982 - accuracy: 0.6222
Epoch 6/100
2/2 [==============================] - 3s 811ms/step - loss: 0.8145 - accuracy: 0.6444
Epoch 7/100
2/2 [==============================] - 3s 817ms/step - loss: 0.7552 - accuracy: 0.7778
Epoch 8/100
2/2 [==============================] - 3s 806ms/step - loss: 0.7615 - accuracy: 0.5778
Epoch 9/100
2/2 [==============================] - 3s 1s/step - loss: 0.7616 - accuracy: 0.7333
Epoch 10/100
2/2 [==============================] - 3s 812ms/step - loss: 0.6610 - accuracy: 0